# Deploying an LLM to Amazon SageMaker AI real-time endpoint

## Prerequisites

To use SageMaker AI endpoints in these examples, you will need to first deploy a managed endpoint. In this example you will deploy an endpoint through SageMaker Jumpstart, a feature that helps machine learning practitioners quickly get started with hundreds of production-ready models in SageMaker AI.

## Dependencies

<div class="alert alert-block alert-info">
⚠️ <b>Important:</b> Make sure you've run the <code>1-required-dependencies.ipynb</code> notebook in this folder before proceeding. If you haven't, close this notebook, run the previous one first, then come back to this.
</div>

## Deploy the model

> Note: skip the cell below if you have already deployed your model.

In [ ]:
import boto3
import time
from datetime import datetime
from sagemaker.core.helper.session_helper import get_execution_role
from sagemaker.core.resources import Model as SageMakerModel, EndpointConfig, Endpoint
from sagemaker.core.shapes import ContainerDefinition, ProductionVariant

def get_role():
    """Get SageMaker execution role."""
    try:
        return get_execution_role()
    except:
        return input("Enter your SageMaker role ARN: ").strip()

role = get_role()
region = boto3.Session().region_name
sm_client = boto3.client('sagemaker')

model_id = "Qwen/Qwen3-4B"
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
model_name = f"qwen3-4b-{timestamp}"
endpoint_name = f"qwen3-4b-ep-{timestamp}"

# DJL LMI container with vLLM
image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.34.0-lmi16.0.0-cu128-v1.2"

# Create container definition with environment variables
container = ContainerDefinition(
    image=image_uri,
    environment={
        "HF_MODEL_ID": model_id,
        "OPTION_MAX_MODEL_LEN": f"{1024*16}",
        "OPTION_QUANTIZE": "fp8",
        "OPTION_DTYPE": "bf16",
        "SERVING_FAIL_FAST": "true",
        "OPTION_ROLLING_BATCH": "disable",
        "OPTION_ASYNC_MODE": "true",
        "OPTION_ENTRYPOINT": "djl_python.lmi_vllm.vllm_async_service",
        "OPTION_ENABLE_AUTO_TOOL_CHOICE": "true",
        "OPTION_TOOL_CALL_PARSER": "hermes",
        "OPTION_ENABLE_REASONING": "true",
        "OPTION_REASONING_PARSER": "qwen3",
    }
)

# Create model
print(f"Creating model: {model_name}")
sm_model = SageMakerModel.create(
    model_name=model_name,
    primary_container=container,
    execution_role_arn=role,
)

# Create endpoint config
print(f"Creating endpoint config: {endpoint_name}")
endpoint_config = EndpointConfig.create(
    endpoint_config_name=endpoint_name,
    production_variants=[
        ProductionVariant(
            variant_name='AllTraffic',
            model_name=model_name,
            initial_instance_count=1,
            instance_type='ml.g5.xlarge',
            initial_variant_weight=1.0,
        )
    ]
)

# Create endpoint
print(f"Creating endpoint: {endpoint_name}")
endpoint = Endpoint.create(
    endpoint_name=endpoint_name,
    endpoint_config_name=endpoint_name,
)

# Wait for endpoint to be ready
print("Waiting for endpoint to be InService (5-7 minutes)...")
while True:
    status = sm_client.describe_endpoint(EndpointName=endpoint_name)['EndpointStatus']
    print(f"  Status: {status}")
    if status == 'InService':
        print("Endpoint is ready!")
        break
    elif status == 'Failed':
        raise Exception("Endpoint deployment failed")
    time.sleep(30)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
----------------------!

In [2]:
SAGEMAKER_ENDPOINT_NAME = endpoint_name
print(f"Endpoint name: {SAGEMAKER_ENDPOINT_NAME}")

%store SAGEMAKER_ENDPOINT_NAME

Endpoint name: Qwen3-4B-ep-2025-10-15-17-49-51-128
Stored 'SAGEMAKER_ENDPOINT_NAME' (str)


<div class="alert alert-block alert-info">
⚠️ <b>Note:</b> deployment will take 5~7 minutes. Take note of the endpoint name and the inference component names, as they will be needed later.
</div>